<h2></h2>
<h1 align='center'><b>Main Code - Código principal</b></h1>
<h2 align='center'>Movie Analysis</h2>

<p align='center'>
<img alt="stack" src="img/portada.jpg" width="75%">
</p>

Importamos Librerías - Import Libraries

In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

Importamos DataSet - Import Dataset

In [3]:
df = pd.read_csv("movies_dataset.csv", delimiter=",",index_col=False, names=["adult","belongs_to_collection","budget","genres","homepage","id","imdb_id","original_language","original_title","overview","popularity","poster_path","production_companies","production_countries","release_date","revenue","runtime","spoken_languages","status","tagline","title","video","vote_average","vote_count"],header=0)
df = df.drop(columns=["video","imdb_id","adult","original_title","vote_count","poster_path", "homepage"])

##### - Some rows have string data, so we need to convert them in datatype list

##### - Algunas columnas tienen datos string por ende hay que traducirlos a listas

In [4]:
def get_values(lista):
    if type(lista) == float:
        return ""
    res=""
    import json
    lista_diccionarios = json.loads(lista.replace("'", "\""))
    for dic in lista_diccionarios:
        res+=str((list(dic.values())[1]))+", "
    return res[:(len(res))-2]

def get_values_countries(lista):
    if type(lista) == float:
        return ""
    if lista == "":
        return ""
    res=""
    import json
    lista_diccionarios = json.loads(lista.replace("'", "\""))
    
    if type(lista_diccionarios)== float:
        return ""
    
    for dic in lista_diccionarios:
        res+=str((list(dic.values())[0]))+", "
    return res[:(len(res))-2]

import json
count=0
unalista=[]
for i, val in enumerate(df["production_companies"]):
    if isinstance(val, str):
        try:
            json.loads(val.replace("\"","").replace("'", "\""))
        except json.JSONDecodeError as e:
            unalista.append(i)
unalista = set(unalista)

def get_values_production_companies_alter(lista):
    import re
    regex = r"'name': '(?:(?<!\\)'|\\')*([^']+)'"
    matches = re.findall(regex, lista)
    
    if not matches:
        return ""
    
    return ", ".join(matches)

def get_value_production_companies(x):
    if x.name in unalista:
        return x["production_companies"]
    if type(x["production_companies"]) != str:
        return x["production_companies"]
    if "\'" in x["production_companies"]:
        return get_values_production_companies_alter(x["production_companies"])
    
    res=""
    import json
    lista_diccionarios = json.loads(x["production_companies"].replace("'", "\""))
    for dic in lista_diccionarios:
        res+=str((list(dic.values())[1]))+", "
    return res[:(len(res))-2]

def get_collection_name(string):
    if type(string) != str:
        return string
    v1=3
    res=""
    for let in string:
        if let == " ":
            v1-=1
        if v1 <= 0 and let == ",":
            break
        if v1 <= 0:
            res+=let
    return res

def get_year(lista):
    return lista[:4]

def return_fun(rev,bud):
    if bud==0:
        return 0
    return rev/bud

df["genres"]=df["genres"].apply(get_values)
df["belongs_to_collection"]=df["belongs_to_collection"].apply(get_collection_name)
df["production_companies"] = df.apply(get_value_production_companies, axis=1)

listaa=[]
for i, val in enumerate(df["production_countries"]):
    if isinstance(val, str):
        try:
            json.loads(val.replace("\"","").replace("'", "\""))
        except json.JSONDecodeError as e:
            listaa.append(i)


df["production_countries"] = df["production_countries"].drop(index=listaa)
df["production_countries"]=df["production_countries"].apply(get_values_countries)

listaa=[]
for i, val in enumerate(df["spoken_languages"]):
    if isinstance(val, str):
        try:
            json.loads(val.replace("\"","").replace("'", "\""))
        except json.JSONDecodeError as e:
            listaa.append(i)

df["spoken_languages"] = df["spoken_languages"].drop(index=listaa)
df["spoken_languages"]=df["spoken_languages"].apply(get_values_countries)

Data Cleaning - Limpeiza de Datos

In [5]:
df["budget"] = df["budget"].astype(int)
df = df.dropna(subset=['release_date'])

df["release_year"]= df['release_date'].apply(get_year)
nulos = df["release_year"].isnull().sum()

df["revenue"] = df["revenue"].apply(int)
df["budget"] = df["budget"].apply(int)

df['belongs_to_collection'] = df['belongs_to_collection'].fillna('')
df['production_countries']=df['production_countries'].replace(" ","UnknownCountry")
df['production_countries']=df['production_countries'].replace("","UnknownCountry")
df['genres']=df['genres'].replace(" ","UnknownGenre")
df['genres']=df['genres'].replace("","UnknownGenre")

df["return"]=df['revenue']/df['budget']

df['release_date'] = pd.to_datetime(df['release_date'])

## Descriptive Functions - Funciónes Descriptivas

#### peliculas_mes

##### - Se ingresa el mes y la funcion retorna la cantidad de peliculas que se estrenaron ese mes (nombre del mes, en str, ejemplo 'enero') historicamente

##### - The month is entered and the function returns the number of movies that were released that month (name of the month, in str, example 'January') historically

In [6]:
def peliculas_mes(mes): 
    cant=0
    if mes == "enero":
        mesNum=1
    elif mes == "febrero":
        mesNum = 2
    elif mes == "marzo":
        mesNum = 3 
    elif mes == "abril":
        mesNum = 4 
    elif mes == "mayo":
        mesNum = 5 
    elif mes == "junio":
        mesNum = 6
    elif mes == "julio":
        mesNum = 7 
    elif mes == "agosto":
        mesNum = 8 
    elif mes == "septiembre":
        mesNum = 9 
    elif mes == "octubre":
        mesNum = 10 
    elif mes == "noviembre":
        mesNum = 11 
    elif mes == "diciembre":
        mesNum = 12 
    else:
        return print("Ingresa un mes correcto") 
    
    for row, mon in enumerate(df['release_date']):
        try:
            df['release_date'][row]
        except KeyError:
            continue

        if df['release_date'][row].month == mesNum:
            cant = cant + 1

    return 'En el mes de ' + str(mes)  + ' se han producido ' + str(cant) + " películas"

#### peliculas_dia

##### - Se ingresa el dia y la funcion retorna la cantidad de peliculas que se estrenaron ese dia (de la semana, en str, ejemplo 'lunes') historicamente

##### - The day is entered and the function returns the number of movies that were released that day (of the week, in str, example 'Monday') historically

In [7]:
def peliculas_dia(dia):  
    cant=0
    if dia == "martes":
        diaNum=1
    elif dia == "miercoles":
        diaNum = 2
    elif dia == "jueves":
        diaNum = 3 
    elif dia == "viernes":
        diaNum = 4 
    elif dia == "sabado":
        diaNum = 5 
    elif dia == "domingo":
        diaNum = 6
    elif dia == "lunes":
        diaNum = 0 
    else:
        return print("Ingresa un dia de la semana correcto") 

    for row, d in enumerate(df['release_date']):
        try:
            df['release_date'][row]
        except KeyError:
            continue

        if df['release_date'][row].weekday() == diaNum:
            cant = cant + 1
    
    return 'El día ' + str(dia)  + ' han producido ' + str(cant) + " películas"

#### franquicia

##### - Se ingresa la franquicia, retornando la cantidad de peliculas, ganancia total y promedio

##### - The franchise is entered, returning the number of movies, total and average profit

In [8]:
def franquicia(franquicia):  
    if franquicia=="":
       return print("Ingresa una franquicia") 
    cant,total,ctrl=0,0,False
    for row, d in enumerate(df['belongs_to_collection']):
        try:
            df['belongs_to_collection'][row]
        except KeyError:
            continue

        else:
            if franquicia in df['belongs_to_collection'][row]:
                cant = cant + 1
                total+=df['revenue'][row]
                ctrl=True
    if total == 0:
        cant = 1

    if not ctrl:
        return print('La franquicia no se encuentra en la lista')
    
    return 'La franquicia de ' + str(franquicia) + ' produció una cantidad de ' + str(cant) + ' películas con una ganancia promedio total de '+ str(int(total/cant))

#### peliculas_pais

##### - Ingresas el pais, retornando la cantidad de peliculas producidas en el mismo

##### - Enter the country, returning the number of films produced in the same

In [9]:
def peliculas_pais(pais):
    if pais=="":
       return print("Ingresa un pais") 
    cant,ctrl=0,False
    for row, p in enumerate(df['production_countries']):
        try:
            df['production_countries'][row]
        except KeyError:
            continue
        else:
            if pais in df['production_countries'][row]:
                cant = cant + 1
                ctrl=True
    if not ctrl:
        return print('El pais no se encuentra en la lista')
    return 'El país de '+ str(pais) + ' produció una cantidad de '+ str(cant) + " películas"

#### productoras

##### - Ingresas la productora, retornando la ganancia total y la cantidad de peliculas que produjeron

##### - You enter the production company, returning the total profit and the number of movies they produced

In [10]:
def productoras(productora): 
    if productora=="":
       return print("Ingresa una productora") 
    cant,total,ctrl=0,0,False
    for row, p in enumerate(df['production_companies']):
        try:
            df['production_companies'][row]
        except KeyError:
            continue
        else:
            if productora in df['production_companies'][row]:
                cant = cant + 1
                total+=df['revenue'][row]
                ctrl=True
    if not ctrl:
        return print('La productora no se encuentra en la lista')
    if total == 0:
        total="( Desconocido )"

    return "La productora " + str(productora)  +' gano un total de ' + str(total) + ' dolares, con una cantidad de ' + str(cant) + " películas"

#### - Generar una lista de géneros (únicos) y una lista de paises (únicos)

#### - Generate a list of genres (unique) and a list of countries (unique)

In [11]:
list_genres=[]
list_pais=[]
for row in df["genres"]: 
    word=""
    for let in row:
        if let==",":
            if word not in list_genres:
                list_genres.append(word)
            word=""
        elif let == " ":
            continue
        else:
            word+=let

for row in df["production_countries"]: 
    word=""
    for let in row:
        if let==",":
            if word not in list_pais:
                list_pais.append(word)
            word=""
        elif let == " ":
            continue
        else:
            word+=let

## Machine Learning

#### - "dataMachine" es una copia del dataframe anterior pero tendrá ciertas modificaciones para su proceso con inteligencia artificial

#### - "dataMachine" is a copy of the previous dataframe but will have certain modifications for its processing with artificial intelligence

In [12]:
dataMachine=pd.DataFrame()
dataMachine["title"] = df["title"]
dataMachine["vote_average"] = df["vote_average"]
dataMachine["runtime"] = df["runtime"]
dataMachine["release_year"] = df["release_year"]
dataMachine["genres"] = df["genres"]
dataMachine["production_countries"] = df["production_countries"]

In [13]:
countries = dataMachine['production_countries'].str.split(', ', expand=True)
genres = dataMachine['genres'].str.split(', ', expand=True)

#### - Se convierten las columnas categóricas a dummies

#### - Categorical columns are converted to dummies

In [14]:
country_dummies = pd.get_dummies(countries, prefix='', prefix_sep='')
genres_dummies = pd.get_dummies(genres, prefix='', prefix_sep='')

country_dummies = country_dummies.groupby(level=0, axis=1).sum().clip(upper=1)
genres_dummies = genres_dummies.groupby(level=0, axis=1).sum().clip(upper=1)

# Agregar las columnas de dummies al dataframe original
dataMachine = pd.concat([dataMachine, genres_dummies], axis=1)
dataMachine = pd.concat([dataMachine, country_dummies], axis=1)

# Eliminar la columna original de 'production_countries'
dataMachine.drop('production_countries', axis=1, inplace=True)
dataMachine.drop('genres', axis=1, inplace=True)

In [15]:
dataMachine["runtime"] = dataMachine["runtime"].fillna(0)
dataMachine["release_year"] = dataMachine["release_year"].fillna(0)
dataMachine["vote_average"] = dataMachine["vote_average"].fillna(0)
dataMachine['runtime'] = dataMachine['runtime'].apply(int)

Eliminar las filas donde runtime > 200 and < 60

Delete the rows where runtime > 200 and < 60

In [16]:
dataMachine = dataMachine.drop(dataMachine[dataMachine['runtime'] > 200].index)
dataMachine = dataMachine.drop(dataMachine[dataMachine['runtime'] < 60].index)

In [17]:
dataMachine['runtime'] = dataMachine['runtime'].replace(0,int(dataMachine['runtime'].mean()))
dataMachine['vote_average'] = dataMachine['vote_average'].replace(0,int(dataMachine['vote_average'].mean()))
dataMachine['release_year'] = dataMachine['release_year'].apply(int)

#### - Algunos títulos estaban repetidos por el hecho de que hay varias películas con el mismo nombre pero distinto año de lanzamiento#, asi que se le agrega el año al nombre

#### - Some titles were repeated due to the fact that there are several movies with the same name but different years of release, so the year is added to the name

In [18]:
lista_duplicados = []
for n in dataMachine[dataMachine.duplicated(subset=["title"], keep=False)]["title"]:
    if n not in lista_duplicados:
        lista_duplicados.append(n)

def add_release_year(title, release_year):
    if title in lista_duplicados:
        return f"{title} ({release_year})"
    else:
        return title

dataMachine["title"] = dataMachine.apply(lambda x: add_release_year(x["title"], x["release_year"]), axis=1)

### Machine learning - Kmeans - Clustering

In [19]:
# seleccionar características

X = dataMachine[["runtime","release_year"]]

# crear objeto StandardScaler y aplicarlo a los datos
scaler = StandardScaler()
X_norm = scaler.fit_transform(X)

# reemplazar valores normalizados en el DataFrame original
dataMachine[["runtime","release_year"]] = X_norm

# seleccionar características
X = dataMachine.columns[2:]
# determinar el número de clusters
n_clusters = 15

# crear modelo KMeans
kmeans = KMeans(n_clusters=n_clusters,n_init=10, random_state=42)

# entrenar modelo con datos normalizados
kmeans.fit(X_norm)

# predecir cluster para cada observación
cluster_labels = kmeans.predict(X_norm)

# agregar etiquetas de cluster a los datos originales
dataMachine["cluster"] = cluster_labels

# función para obtener películas similares

#dataMachine[["runtime","release_year"]] = memory
def get_similar_movies(title):
    # encontrar cluster de la película buscada
    try:
        movie_cluster = dataMachine[dataMachine["title"] == title]["cluster"].iloc[0]
    except IndexError:
        return print("La pelicula que ingresaste no esta en la lista o tiene un error ortografico")
    
    movie_cluster = dataMachine[dataMachine["title"] == title]["cluster"].iloc[0]
    
    # seleccionar películas del mismo cluster, excepto la buscada
    similar_movies = dataMachine[(dataMachine["cluster"] == movie_cluster) & (dataMachine["title"] != title)].copy()
    
    # calcular distancia euclidiana entre cada película y la de referencia
    reference_movie = dataMachine[dataMachine["title"] == title][X].values
    distances = np.linalg.norm(similar_movies[X].values - reference_movie, axis=1)
    
    # agregar distancia como columna en similar_movies y ordenar por distancia
    similar_movies["distance"] = distances
    similar_movies = similar_movies.sort_values(by=["distance"], ascending=[True])
    
    tabla_resultado = pd.DataFrame(similar_movies.head(5)["title"]).reset_index(drop=True)
    tabla_resultado.index = tabla_resultado.index + 1

    return tabla_resultado

#### - Guardar en un csv la lista de películas

#### - Save in a csv file the list of movies

In [20]:
# Obtener la lista de títulos de películas
movies_list = dataMachine["title"].unique().tolist()

# Convertir la lista a un DataFrame de pandas
data_frame = pd.DataFrame(movies_list, columns=["title"])

# Escribir el DataFrame en el archivo CSV
data_frame.to_csv("movies_list.csv", index=False)


<h2></h2>
<h2 align='center'><b>Final Results - Resultados Finales</b></h2>
<h2></h2>

### - Ejemplo FINAL de la recomendación

In [21]:
get_similar_movies("The Godfather")

,title
1,Helter Skelter (1976)
2,Prince of the City
3,The Godfather: Part II
4,Out of Africa
5,Casino


### - Ejemplo similar al anterior pero con un input
### - Example similar to the previous one but with an input

In [22]:
movie = input("Ingresa una película para devolver las 5 mas similares")

get_similar_movies(movie)

La pelicula que ingresaste no esta en la lista o tiene un error ortografico


### - Ejemplo de las funciónes secundarias
### - Example of secondary functions

In [23]:
print(peliculas_mes("marzo"))
print("-")
print(peliculas_dia("jueves"))
print("-")
print(franquicia("Toy Story Collection"))
print("-")
print(peliculas_pais("US"))
print("-")
print(productoras("Pixar Animation Studios"))

En el mes de marzo se han producido 3545 películas
-
El día jueves han producido 7504 películas
-
La franquicia de Toy Story Collection produció una cantidad de 3 películas con una ganancia promedio total de 645963535
-
El país de US produció una cantidad de 21125 películas
-
La productora Pixar Animation Studios gano un total de 11188533734 dolares, con una cantidad de 52 películas


### - Guardar los datos procesados para su posterior análisis estadístico
### - Save processed data for later statistical analysis

In [24]:
dataStat = df.drop(columns=["belongs_to_collection","budget","id","revenue","status","release_year","return","spoken_languages","overview","popularity"])
dataStat.to_csv("dataStat.csv",index=False, sep=";")